In [1]:
import HelperFuncs as hfuncs
import numpy as np

We're going to first try training a CNN on the individual images.
We will be using binary cross entropy across the 17 regions.

In [2]:
BATCH_SIZE = 20
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17

#Define a generator function
def myGenerator():
    #AWS and Directory information 
    bucketName = 'miscdatastorage'
    dataDir = 'DHSData/'
    temp_dir = 'temp'
    labels_dir = r'stage1_labels.csv'
    #Connect to AWS
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(bucketName)
    #Initialize required parameters
    key_ary = hfuncs.GetShuffledKeys(bucket)
    labels_dict = hfuncs.GetLabelsDict(labels_dir)
    extension = '.a3daps'
    
    #Initialize AWS Batch Requester
    batchrequester = hfuncs.BatchRequester(bucket,key_ary,labels_dict,dataDir,temp_dir,extension)
    
    #Preprocessing parameters
    n_samples = 50 #Distinct samples (x64 images each) to retrieve iteratively
    
    
    #While there is data left, yield batch
    while batchrequester.DoItemsRemain():
        #Request data
        print("Retrieving data..")
        X,y = batchrequester.NextBatch(n_samples)
        n_angles = X.shape[3] #num angles (64)
        
        #Create efficient mapping for mixing and indexing batch data
        indexing_dict = {}
        order = np.arange(X.shape[0]*n_angles)
        np.random.shuffle(order)
        k = 0
        for s in range(X.shape[0]):
            for a in range(n_angles):
                indexing_dict[order[k]]=[s,a]
                k+=1
            
        print("Data retrieved and indexing computed.")
        
        #Initialize output arrays
        print("Initializing arrays...")
        X_train = np.zeros((X.shape[0]*n_angles,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
        y_train = np.zeros((X.shape[0]*n_angles,ZONES))
        print("Arrays initialized")
    
        #Set counter to 0, channel to 1
        chan = 0 #No need to iterate here
        i = 0
        #Clean each image and store it in output
        while i < X.shape[0] * n_angles:
            j = i
            while j < i+BATCH_SIZE:
                s,a = indexing_dict[j]                    
                X_train[j,:,:,chan] = hfuncs.CropCleanResize(X[s,:,:,a],FINAL_WIDTH,FINAL_HEIGHT)
                y_train[j,:] = y[s,:]
                j += 1
                                    
            yield X_train[i:i+BATCH_SIZE,:,:,:],y_train[i:i+BATCH_SIZE]
            i += BATCH_SIZE

        
        
        



    

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D , AveragePooling2D,Flatten
from keras.models import Model
from keras.layers.core import Dropout
import keras

#Build Basic model

input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))

pooling_1 = MaxPooling2D((3,2),padding='same')(input_img)

tower_1 = Conv2D(32, (1, 1), padding='same', activation='relu')(pooling_1)
tower_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(32, (1, 1), padding='same', activation='relu')(pooling_1)
tower_2 = Conv2D(32, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((2, 2), strides=(1, 1), padding='same')(pooling_1)
tower_3 = Conv2D(32, (1, 1), padding='same', activation='relu')(tower_3)

output_inception = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)

pooling_2 = MaxPooling2D((3,2),padding='same')(output_inception)
pooling_2 = Dropout(0.10)(pooling_2)

tower_1_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(pooling_2)
tower_1_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1_2)

tower_2_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(pooling_2)
tower_2_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2_2)

tower_3_2 = MaxPooling2D((2, 2), strides=(1, 1), padding='same')(pooling_2)
tower_3_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3_2)

output_inception_2 = keras.layers.concatenate([tower_1_2, tower_2_2, tower_3_2], axis=1)

output_inception_2 = Dropout(0.10)(output_inception_2)
output_inception_2 = MaxPooling2D((3, 2),strides=(3,2), padding='same')(output_inception_2)

conv_3 = Conv2D(128, (1, 1), padding='same', activation='relu')(output_inception_2)
last = Flatten()(conv_3)

#List of independent guesses for each zone
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(last))

out = keras.layers.concatenate(output_nodes)

multi_label_model = Model(input_img, out)


Using TensorFlow backend.


In [ ]:
from datetime import datetime
from keras.callbacks import TensorBoard
from keras.optimizers import SGD
from keras import metrics

x = datetime.today()
stamp = "{}-{}-{}_{}:{}:{}".format(x.year,x.month,x.day,x.hour,x.minute,x.second)
tensorboard = TensorBoard(log_dir="logs/{}".format(stamp))


multi_label_model.compile(optimizer='SGD',
                          metrics=[metrics.binary_accuracy,metrics.binary_crossentropy],
                         loss= 'binary_crossentropy')
gen = myGenerator()
multi_label_model.fit_generator(gen,steps_per_epoch=10,epochs=80,callbacks=[tensorboard])


Epoch 1/80
Retrieving data..
Something went wrong. Skipping 4e2e59c040a6751b0037c15bdf03688b
Something went wrong. Skipping e3169583747cf8cceac6b06925703101
Data retrieved and indexing computed.
Initializing arrays...
Arrays initialized
10/10 [==============================] - 132s - loss: 0.3402 - binary_accuracy: 0.8682 - binary_crossentropy: 0.3402    
Epoch 2/80
10/10 [==============================] - 80s - loss: 0.2841 - binary_accuracy: 0.9179 - binary_crossentropy: 0.2841    
Epoch 3/80
10/10 [==============================] - 80s - loss: 0.2506 - binary_accuracy: 0.9297 - binary_crossentropy: 0.2506    
Epoch 4/80
10/10 [==============================] - 80s - loss: 0.2610 - binary_accuracy: 0.9232 - binary_crossentropy: 0.2610    
Epoch 5/80
10/10 [==============================] - 80s - loss: 0.2821 - binary_accuracy: 0.9129 - binary_crossentropy: 0.2821    
Epoch 6/80
10/10 [==============================] - 80s - loss: 0.2682 - binary_accuracy: 0.9200 - binary_crossentropy

10/10 [==============================] - 82s - loss: 0.3175 - binary_accuracy: 0.8965 - binary_crossentropy: 0.3175    
Epoch 55/80
10/10 [==============================] - 82s - loss: 0.3312 - binary_accuracy: 0.8871 - binary_crossentropy: 0.3312    
Epoch 56/80
10/10 [==============================] - 82s - loss: 0.3240 - binary_accuracy: 0.8909 - binary_crossentropy: 0.3240    
Epoch 57/80
10/10 [==============================] - 82s - loss: 0.3177 - binary_accuracy: 0.8941 - binary_crossentropy: 0.3177    
Epoch 58/80
10/10 [==============================] - 82s - loss: 0.3174 - binary_accuracy: 0.8950 - binary_crossentropy: 0.3174    
Epoch 59/80
10/10 [==============================] - 83s - loss: 0.3209 - binary_accuracy: 0.8894 - binary_crossentropy: 0.3209    
Epoch 60/80
10/10 [==============================] - 83s - loss: 0.3069 - binary_accuracy: 0.8959 - binary_crossentropy: 0.3069    
Epoch 61/80
10/10 [==============================] - 82s - loss: 0.3359 - binary_accurac